In [1]:
import os
import pandas as pd

df_sample = pd.read_table("Z:/Documents/GitHub/toy/data/distance-data.tsv")
class_pair = list()
i = 0
for class1 in range(104):
  for class2 in range(class1):
    class_pair.append([class1, class2, df_sample.scaledPatristicDistance[i]])
    i=i+1
import pandas as pd

In [2]:
import tensorflow as tf

In [6]:
df_test = pd.read_table("ptm_distance_sample.tsv")
df_test

,class1,class2,scaledPatristicDistance
0,1,0,0.032521
1,2,0,0.032521
2,2,1,0.032521
3,3,0,0.032521
4,3,1,0.032521
...,...,...,...
5351,103,98,0.281768
5352,103,99,0.279972
5353,103,100,0.263509
5354,103,101,0.263509


In [18]:
import tensorflow as tf

lookup = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
    toy.get_key_for_class2distance(df_test.class1, df_test.class2),
    tf.constant(df_test.scaledPatristicDistance)
    ),
    default_value = -1
)


batch_size = 64
import random
pred = random.sample(range(104), k = batch_size)
real = random.sample(range(104), k = batch_size)
pred_onehot = tf.one_hot(pred, 104)
real_onehot = tf.one_hot(real, 104)
distance = toy.get_distance_from_two_classes(real, pred , LOOKUPTB=lookup)

ce_loss = tf.keras.losses.CategoricalCrossentropy()

ce_loss(real_onehot, pred_onehot), ce_loss(real_onehot, pred_onehot, sample_weight = distance)


(<tf.Tensor: shape=(), dtype=float32, numpy=15.614405>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.789507>)

In [19]:
import toy

pdce =toy.PatristicDistanceCrossentropy(map_label_to_distance=lookup,
                                    weight_range = [0, 1])


pdce(real_onehot, pred_onehot)
                            

<tf.Tensor: shape=(), dtype=float32, numpy=5.789507>

In [21]:
lookup[get_key_for_class2distance(real, pred)], toy.get_distance_from_two_classes(real, pred, lookup)

(<tf.Tensor: shape=(64,), dtype=float64, numpy=
 array([ 0.4454162, -1.       ,  0.4333629,  0.074513 , -1.       ,
         0.2762509,  0.8686061,  0.1295902,  0.8019138,  0.4632265,
        -1.       ,  0.5058361, -1.       ,  0.4574333, -1.       ,
        -1.       ,  0.723174 , -1.       , -1.       ,  0.776907 ,
        -1.       , -1.       , -1.       , -1.       ,  0.1276943,
        -1.       , -1.       ,  0.0253253,  0.62593  ,  0.2617038,
         0.0735819, -1.       , -1.       ,  0.0759526, -1.       ,
        -1.       , -1.       ,  0.7639538, -1.       ,  0.4961066,
         0.1570701, -1.       , -1.       , -1.       , -1.       ,
         0.1367096,  0.1358449,  0.8709418, -1.       ,  0.1309751,
        -1.       , -1.       ,  0.443297 , -1.       ,  0.0254241,
        -1.       , -1.       , -1.       ,  0.0287918,  0.1301732,
        -1.       , -1.       ,  0.4764389, -1.       ])>,
 <tf.Tensor: shape=(64,), dtype=float64, numpy=
 array([ 0.4454162,  0.510169

In [71]:
rPATH = 'data/'
ls_data = [os.path.join(rPATH, ele) for ele in os.listdir(rPATH)]
print(ls_data)
toy.get_class2distance(ls_data[0]).export()

['data/distance-data.tsv', 'data/h22_df_mms_all.tsv', 'data/ptm_distance_sample.tsv']


(<tf.Tensor: shape=(2600285,), dtype=string, numpy=
 array([b'Casasia <=> Lobelia', b'Betula <=> Krascheninnikovia',
        b'Facelis <=> Prescottia', ..., b'Anisocoma <=> Barbacenia',
        b'Isatis <=> Lewisia', b'Petiveria <=> Pinguicula'], dtype=object)>,
 <tf.Tensor: shape=(2600285,), dtype=float32, numpy=
 array([0.100115 , 0.5262681, 0.6471052, ..., 0.8066629, 0.4062916,
        0.3337169], dtype=float32)>)

In [70]:
df_dist = pd.read_table(ls_data[0])
#df_map = pd.read_table(ls_data)
print(df_dist.columns)

Index(['genusX', 'genusY', 'scaledPatristicDistance'], dtype='object')


In [67]:
v,w, =df_dist.genusX[0], tf.cast(df_dist.genusX, dtype=tf.string)[0]
type(v), type(w)

(str, tensorflow.python.framework.ops.EagerTensor)

In [68]:


def get_key_case_str(TrueClass, PredClass):
    out = tf.strings.reduce_join(
				axis = -1,
				inputs = tf.stack([TrueClass, PredClass], axis = 1),
				separator = ' <=> '
			  )
    return out

In [81]:

mapping1 = tf.lookup.StaticHashTable( ### taxonID => Genus
        tf.lookup.KeyValueTensorInitializer(
            get_key_case_str(df_dist.genusX, df_dist.genusY), 
            tf.constant(df_dist.scaledPatristicDistance, dtype = tf.float32)
            ),
            default_value = -1
        )


In [82]:
df_metadata = pd.read_table('data/h22_df_mms_all.tsv')
mapping2 = toy.map_label_to_class_from_path(ls_data[1], keycol="scientificName",
valcol="genus"
)

In [90]:

def get_label_and_class_from_path(df_PATH, keycol, valcol, convert_key_to_int =True,
    sort_alphabetically = True):
    import pandas as pd
    df = pd.read_table(df_PATH)
    df_by_scif = df.groupby(keycol, as_index= False).first() 
    
    if convert_key_to_int:
      mapping_scif2int = toy.get_map_label_str_to_int(df, keycol, 
                          sort_alphabetically = sort_alphabetically)
      ls_label = [mapping_scif2int[ele] for ele in df_by_scif[keycol]]
    else:
      ls_label = [ele for ele in df_by_scif[keycol]]
    
    ls_class = [ele for ele in df_by_scif[valcol]]
    return ls_label, ls_class


In [ ]:
ls_label, ls_class = get_label_and_class_from_path(ls_data[1], keycol="scientificName",
valcol ="genus")


tf.lookup.StaticHashTable( ### taxonID => Genus
        tf.lookup.KeyValueTensorInitializer(
            tf.constant(list(ls_class), dtype = tf.string), 
            tf.constant(list(ls_label), dtype = tf.int32)
            ),
            default_value = -1
        )